# Developing Basic Agents in LangChain - Interactive Notebook

This notebook provides hands-on exercises to accompany the agents lesson:
- Define tools with input schemas and docstrings
- Build ReAct-style agents
- Use OpenAI tool/function-calling agents
- Add memory and streaming to agents
- Handle parsing errors and retries
- Practice exercises for tool design and agent behavior

Ensure your `.env` has necessary keys (e.g., OPENAI_API_KEY).

## 0) Setup and Imports

In [ ]:
import os
from typing import Optional, Dict, Any
from dotenv import load_dotenv
load_dotenv()

# LLMs, prompts, memory
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.memory import ConversationBufferMemory

# Tools
from langchain_core.tools import tool

# Agent construction (ReAct / tool-calling)
react_available = True
tools_agent_available = True
try:
    from langchain.agents import create_react_agent, create_openai_tools_agent, AgentExecutor
except Exception as e:
    print("Agent constructors not directly available from langchain.agents in this version.")
    print("Error:", e)
    react_available = False
    tools_agent_available = False

# Streaming callbacks
from langchain_core.callbacks import StreamingStdOutCallbackHandler

print("Setup complete.")

## 1) Define Tools
Tools should be pure, safe, with clear docstrings and compact outputs.

In [ ]:
from math import isfinite

@tool
def calculator(expression: str) -> str:
    """Evaluate a basic arithmetic expression like '2 + 3 * 4'."""
    try:
        # WARNING: eval is unsafe for untrusted input. Use a proper expression parser in production.
        result = eval(expression, {"__builtins__": {}}, {})
        if isinstance(result, (int, float)) and isfinite(result):
            return str(result)
        return "calc-error: non-finite result"
    except Exception as e:
        return f"calc-error: {e}"

@tool
def search_local_docs(query: str, corpus: Optional[str] = None) -> str:
    """Fake search over a small in-memory corpus. Provide a short answer from local notes."""
    notes = (corpus or "LangChain supports prompts, chains, agents, and memory for LLM apps.")
    if query.lower() in notes.lower():
        return f"Found mention of '{query}': ... {notes[:120]} ..."
    return "No relevant snippets found."

tools = [calculator, search_local_docs]
print("Tools registered:", [t.name for t in tools])

## 2) Build a ReAct Agent
ReAct alternates between reasoning (Thought), tool use (Action), and observation (Observation).

In [ ]:
llm = None
try:
    llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
except Exception as e:
    print("ChatOpenAI unavailable (check OPENAI_API_KEY):", e)

react_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Use tools when helpful. Think step-by-step."),
    ("human", "{input}")
])

if react_available and llm is not None:
    try:
        agent = create_react_agent(llm=llm, tools=tools, prompt=react_prompt)
        agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
        result = agent_executor.invoke({"input": "What is (12+8)/5? Then check if 'agents' appear in our local notes."})
        print("Agent output:\n", result.get("output"))
    except Exception as e:
        print("ReAct agent execution failed:", e)
else:
    print("ReAct path unavailable in this environment/version.")

### Handle parsing and tool errors gracefully
Use `handle_parsing_errors=True` in `AgentExecutor` to allow the agent to recover from minor format issues.

In [ ]:
if react_available and llm is not None:
    try:
        agent = create_react_agent(llm=llm, tools=tools, prompt=react_prompt)
        safe_exec = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)
        bad = safe_exec.invoke({"input": "Compute 3 * (4 + bad)"})
        print("Safe output:\n", bad.get("output"))
    except Exception as e:
        print("Safe executor encountered an error:", e)
else:
    print("Skipping safe executor: ReAct not available.")

## 3) OpenAI Tool-Calling Agent
Leverage function/tool-calling from OpenAI chat models via LangChain utilities when available.

In [ ]:
if tools_agent_available and llm is not None:
    try:
        tool_prompt = ChatPromptTemplate.from_messages([
            ("system", "You are a precise assistant. Use tools when necessary."),
            ("human", "{input}")
        ])
        tool_agent = create_openai_tools_agent(llm, tools, tool_prompt)
        tool_exec = AgentExecutor(agent=tool_agent, tools=tools, verbose=True)
        out = tool_exec.invoke({"input": "Use the calculator to evaluate (7*9)-8 and tell me the result."})
        print("Tool-calling output:\n", out.get("output"))
    except Exception as e:
        print("OpenAI tools agent execution failed:", e)
else:
    print("OpenAI tools agent path not available.")

## 4) Adding Memory to Agents
Use a conversational prompt and buffer memory to preserve recent context for coherent follow-ups.

In [ ]:
if react_available and llm is not None:
    memory = ConversationBufferMemory(memory_key="chat_history", input_key="input")
    mem_prompt = ChatPromptTemplate.from_messages([
        ("system", "You are helpful. Use the chat history as needed."),
        ("placeholder", "{chat_history}"),
        ("human", "{input}")
    ])
    try:
        mem_agent = create_react_agent(llm=llm, tools=tools, prompt=mem_prompt)
        mem_exec = AgentExecutor(agent=mem_agent, tools=tools, verbose=True, memory=memory)
        print(mem_exec.invoke({"input": "Remember that I like short answers."})["output"])
        print(mem_exec.invoke({"input": "What did I say about answer style?"})["output"])
    except Exception as e:
        print("Memory-enabled agent failed:", e)
else:
    print("Skipping memory integration: ReAct or LLM unavailable.")

## 5) Streaming Agent Responses
Stream tokens for better UX and observability during longer responses or multi-step tool use.

In [ ]:
try:
    stream_llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0, streaming=True, callbacks=[StreamingStdOutCallbackHandler()])
except Exception as e:
    stream_llm = None
    print("Streaming LLM unavailable:", e)

if react_available and stream_llm is not None:
    try:
        stream_agent = create_react_agent(llm=stream_llm, tools=tools, prompt=react_prompt)
        stream_exec = AgentExecutor(agent=stream_agent, tools=tools, verbose=True)
        _ = stream_exec.invoke({"input": "Show your steps and compute 17*19 using the calculator."})
    except Exception as e:
        print("Streaming agent failed:", e)
else:
    print("Streaming path not available.")

## 6) Exercises
A) Calculator + Units Agent
- Implement a `unit_convert(value: float, from_unit: str, to_unit: str)` tool for km<->miles.
- Build an agent that decides between `calculator` or `unit_convert`.
- Test: "What is 150 miles in km plus 12?".

B) Local Notes QA Agent
- Add a `load_notes(topic: str)` tool that reads a snippet from `./notes/*.txt`.
- Combine with `search_local_docs` to answer: "Find any mention of 'vector memory' and summarize in one sentence.".

C) Robustness
- Use `handle_parsing_errors=True` in `AgentExecutor`.
- Simulate a tool error and show a graceful fallback message.

D) Memory-Aware Agent
- Integrate `ConversationBufferMemory` and verify follow-up consistency.

E) Observability
- Log timing and tool inputs/outputs; print a minimal trace after each run.

### Exercise Scaffold: Unit Conversion Tool and Agent

In [ ]:
from dataclasses import dataclass

@tool
def unit_convert(value: float, from_unit: str, to_unit: str) -> str:
    """Convert simple units, currently supports miles<->km (case-insensitive)."""
    conv = {
        ("miles", "km"): 1.60934,
        ("km", "miles"): 1/1.60934,
    }
    try:
        k = (from_unit.strip().lower(), to_unit.strip().lower())
        if k not in conv:
            return "convert-error: unsupported units"
        return str(value * conv[k])
    except Exception as e:
        return f"convert-error: {e}"

exercise_tools = [calculator, unit_convert]
print("Exercise tools:", [t.name for t in exercise_tools])

if react_available and llm is not None:
    try:
        ex_prompt = ChatPromptTemplate.from_messages([
            ("system", "You are a careful assistant. Prefer calculator for arithmetic and unit_convert for unit changes."),
            ("human", "{input}")
        ])
        ex_agent = create_react_agent(llm=llm, tools=exercise_tools, prompt=ex_prompt)
        ex_exec = AgentExecutor(agent=ex_agent, tools=exercise_tools, verbose=True, handle_parsing_errors=True)
        # Try: 150 miles to km, then +12
        out = ex_exec.invoke({"input": "Convert 150 miles to km and add 12 to the result."})
        print("Exercise agent output:\n", out.get("output"))
    except Exception as e:
        print("Exercise agent failed:", e)
else:
    print("Skipping exercise agent construction.")

## 7) Observability: Minimal Tracing for Tool Calls
Create a wrapper to log calls and durations of each tool invocation.

In [ ]:
import time
from functools import wraps

def log_tool(fn):
    @wraps(fn)
    def wrapper(*args, **kwargs):
        start = time.time()
        try:
            result = fn(*args, **kwargs)
            return result
        finally:
            dur = (time.time() - start) * 1000
            print(f"[tool] {fn.__name__} took {dur:.1f} ms")
    return wrapper

# Wrap raw callables if needed (LangChain tool decorators already generate Tool objects)
wrapped_calc = log_tool(calculator.func)
print("2+2 via wrapped calc:", wrapped_calc("2+2"))

## Summary
You built tools and agents (ReAct and OpenAI tool-calling), added memory and streaming, and handled parsing errors. Complete the exercises to deepen your understanding and adapt these patterns for your applications.